In [1]:
import numpy as np
import os
import sys
import cv2
import matplotlib.pyplot as plt
import open3d as o3d
%matplotlib inline 

src_path = os.path.abspath("../..")
if src_path not in sys.path:
    sys.path.append(src_path)
%load_ext autoreload
from dataset.kitti_odometry_dataset import KittiOdometryDataset, KittiOdometryDatasetConfig
from dataset.filters.filter_list import FilterList
from dataset.filters.kitti_gt_mo_filter import KittiGTMovingObjectFilter
from dataset.filters.range_filter import RangeFilter
from dataset.filters.apply_pose import ApplyPose

import scipy
from scipy.spatial.distance import cdist
from normalized_cut import normalized_cut
from ncuts_utils import ncuts_chunk,kDTree_1NN_feature_reprojection_colors, get_merge_pcds
from dataset_utils import * 
from point_cloud_utils import get_pcd, transform_pcd, kDTree_1NN_feature_reprojection, remove_isolated_points, get_subpcd, get_statistical_inlier_indices, merge_chunks_unite_instances
from aggregate_pointcloud import aggregate_pointcloud
from visualization_utils import generate_random_colors, color_pcd_by_labels,generate_random_colors_map
from sam_label_distace import sam_label_distance
from chunk_generation import subsample_positions, chunks_from_pointcloud, indices_per_patch, tarl_features_per_patch, image_based_features_per_patch, dinov2_mean, get_indices_feature_reprojection
from metrics_class import Metrics

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
import sklearn

def intersect(pred_indices, gt_indices):
        intersection = np.intersect1d(pred_indices, gt_indices)
        return intersection.size / pred_indices.shape[0]


def remove_semantics(labels,preds):
        gt_idcs = np.where(labels == 0)[0]
        new_ncuts_labels = preds.copy()
        for i in np.unique(preds):
                pred_idcs = np.where(preds == i)[0]
                cur_intersect = intersect(pred_idcs,gt_idcs)
                if cur_intersect > 0.8:
                        new_ncuts_labels[pred_idcs] = 0
        return new_ncuts_labels



def color_pcd_by_labels(pcd, labels,colors=None,gt_labels=None):
    
    if colors == None : 
        colors = generate_random_colors(2000)
    pcd_colored = copy.deepcopy(pcd)
    pcd_colors = np.zeros(np.asarray(pcd.points).shape)
    if gt_labels is None :
    	unique_labels = list(np.unique(labels)) 
    else: 
        unique_labels = list(np.unique(gt_labels))
    
    background_color = np.array([0,0,0])


    #for i in range(len(pcd_colored.points)):
    for i in unique_labels:
        if i == -1 : 
            continue
        idcs = np.where(labels == i)
        idcs = idcs[0]
        if i == 0 : 
            pcd_colors[idcs] = background_color
        else : 
            try : 
                pcd_colors[idcs] = np.array(colors[unique_labels.index(i)])
            except : 
                import pdb; pdb.set_trace()
        
        #if labels[i] != (-1):
        #    pcd_colored.colors[i] = np.array(colors[labels[i]]) / 255
    pcd_colored.colors = o3d.utility.Vector3dVector(pcd_colors/ 255)
    return pcd_colored


def get_merge_pcds(out_folder_ncuts):
        point_clouds = []

        # List all files in the folder
        files = os.listdir(out_folder_ncuts)
        files.sort()

        # Filter files with a .pcd extension
        pcd_files = [file for file in files if file.endswith(".pcd")]
        print(pcd_files)
        # Load each point cloud and append to the list
        for pcd_file in pcd_files:
                file_path = os.path.join(out_folder_ncuts, pcd_file)
                point_cloud = o3d.io.read_point_cloud(file_path)
                point_clouds.append(point_cloud)
        return point_clouds

def uniform_down_sample_with_indices(points,every_k_points):
    # Create a new point cloud for the downsampled output
    
    # List to hold the indices of the points that are kept
    indices = []
    
    
    # Iterate over the points and keep every k-th point
    for i in range(0, points.shape[0], every_k_points):
        indices.append(i)

    
    return indices


def merge_unite_gt(chunks):
    last_chunk = chunks[0] 
    merge = o3d.geometry.PointCloud()
    merge += last_chunk

    for new_chunk in chunks[1:]:
        merge += new_chunk
    
    merge.remove_duplicated_points()
    return merge 

def downsample_chunk(points,chunk_preds_ncut,kitti_chunk_labels):
        num_points_to_sample = 60000
        every_k_points = int(kitti_chunk_labels.shape[0] /num_points_to_sample) 
        indeces = uniform_down_sample_with_indices(kitti_chunk_labels,every_k_points)

        points = points[indeces]
        chunk_preds_ncut = chunk_preds_ncut[indeces]
        kitti_chunk_labels = kitti_chunk_labels[indeces]
        return points,chunk_preds_ncut,kitti_chunk_labels

def divide_indices_into_chunks(max_index, chunk_size=1000):
    chunks = []
    for start in range(0, max_index, chunk_size):
        end = min(start + chunk_size, max_index)
        chunks.append((start, end))
    return chunks


Here we define the dataset depending on kitti sequence!

In [3]:
DATASET_PATH = os.path.join('/media/cedric/Datasets1/semantic_kitti/')
import shutil 
import gc 

minor_voxel_size = 0.05
major_voxel_size = 0.35
chunk_size = np.array([25, 25, 25]) #meters
overlap = 24 #meters
ground_segmentation_method = 'patchwork' 
NCUT_ground = False 
out_folder_ncuts = 'test_data/'

out_folder = 'pcd_preprocessed/'
if os.path.exists(out_folder) == False : 
        os.makedirs(out_folder)

alpha = 1.0
theta = 0.5
colors = generate_random_colors_map(600)
beta = 0.0
gamma = 0.0
proximity_threshold = 1.0

seqs = list(range(0,2))

for seq in seqs : 
        print("Sequence",seq)
        SEQUENCE_NUM = seq
        dataset = create_kitti_odometry_dataset(DATASET_PATH,SEQUENCE_NUM,ncuts_mode=True)
        chunks_idcs = divide_indices_into_chunks(len(dataset))
        
        
        data_store_folder = out_folder + str(SEQUENCE_NUM) + "/"
        if os.path.exists(data_store_folder) == False :
                        os.makedirs(data_store_folder)
        print("STORE FOLDER",data_store_folder)
        
        for cur_idx,cidcs in enumerate(chunks_idcs):
                #if cur_idx == 0 : 
                #         continue
        
                ind_start = cidcs[0]
                ind_end = cidcs[1]
                
                print('ind start',ind_start)
                print('ind end',ind_end)
                
                if os.path.exists(f'{out_folder}non_ground{SEQUENCE_NUM}_{cur_idx}.pcd') == False : 
                        print('process poses')
                        process_and_save_point_clouds(dataset,ind_start,ind_end,minor_voxel_size=minor_voxel_size,
                                                major_voxel_size=major_voxel_size,icp=False,
                                                out_folder=out_folder,sequence_num=SEQUENCE_NUM,
                                                ground_segmentation_method=ground_segmentation_method,cur_idx=cur_idx)
                
                if os.path.exists(f'{out_folder}pcd_nonground_minor{SEQUENCE_NUM}_{cur_idx}.pcd') == False : 
                #    if True == True : 
                        print('load and downsample points')
                        pcd_ground_minor, pcd_nonground_minor,\
                        all_poses, T_pcd, first_position,kitti_labels = load_and_downsample_point_clouds(out_folder,SEQUENCE_NUM,minor_voxel_size,\
                                                                                ground_mode=ground_segmentation_method,cur_idx=cur_idx)
                        
                        print('write pcds')
                        print(pcd_ground_minor)
                        o3d.io.write_point_cloud(f'{out_folder}pcd_ground_minor{SEQUENCE_NUM}_{cur_idx}.pcd', pcd_ground_minor, write_ascii=False, compressed=False, print_progress=True)
                        o3d.io.write_point_cloud(f'{out_folder}pcd_nonground_minor{SEQUENCE_NUM}_{cur_idx}.pcd', pcd_nonground_minor, write_ascii=False, compressed=False, print_progress=True)
                        print('write labels')
                        np.savez(f'{out_folder}kitti_labels_preprocessed{SEQUENCE_NUM}_{cur_idx}.npz',panoptic_nonground=kitti_labels['panoptic_nonground'],
                                                                panoptic_ground=kitti_labels['panoptic_ground'],
                                                                instance_nonground=kitti_labels['instance_nonground'],
                                                                instance_ground=kitti_labels['instance_ground'],
                                                                seg_ground = kitti_labels['seg_ground'],
                                                                seg_nonground=kitti_labels['seg_nonground']
                                                                )
                print("load pcd")
                pcd_ground_minor = o3d.io.read_point_cloud(f'{out_folder}pcd_ground_minor{SEQUENCE_NUM}_{cur_idx}.pcd')
                pcd_nonground_minor = o3d.io.read_point_cloud(f'{out_folder}pcd_nonground_minor{SEQUENCE_NUM}_{cur_idx}.pcd')
                
                print("load data")
                kitti_labels_orig = {}
                with np.load(f'{out_folder}kitti_labels_preprocessed{SEQUENCE_NUM}_{cur_idx}.npz') as data :
                        kitti_labels_orig['panoptic_ground'] = data['panoptic_ground']
                        kitti_labels_orig['panoptic_nonground'] = data['panoptic_nonground']
                        kitti_labels_orig['instance_ground'] = data['instance_ground']
                        kitti_labels_orig['instance_nonground'] = data['instance_nonground']
                        kitti_labels_orig['seg_nonground'] = data['seg_nonground']
                        kitti_labels_orig['seg_ground'] = data['seg_ground']
        
                with np.load(f'{out_folder}all_poses_' + str(SEQUENCE_NUM) + '_' + str(cur_idx) + '.npz') as data:
                        all_poses = data['all_poses']
                        T_pcd = data['T_pcd']
                        first_position = T_pcd[:3, 3]
                        
                print("pose downsample")
                if os.path.exists(f'{out_folder}subsampled_data{SEQUENCE_NUM}_{cur_idx}.npz') == False : 
                        poses, positions, \
                        sampled_indices_local, sampled_indices_global = subsample_and_extract_positions(all_poses,ind_start=ind_start,sequence_num=SEQUENCE_NUM,out_folder=out_folder,cur_idx=cur_idx)
                
                with np.load(f'{out_folder}subsampled_data{SEQUENCE_NUM}_{cur_idx}.npz') as data:
                        poses = data['poses']
                        positions = data['positions']
                        sampled_indices_local = list(data['sampled_indices_local'])
                        sampled_indices_global = list(
                         data['sampled_indices_global'])
                        
                
                
                print("chunk downsample")
                pcd_nonground_chunks, pcd_ground_chunks,\
                pcd_nonground_chunks_major_downsampling, pcd_ground_chunks_major_downsampling, \
                indices,indices_ground, center_positions, \
                center_ids, chunk_bounds, kitti_labels = chunk_and_downsample_point_clouds(pcd_nonground_minor, pcd_ground_minor, T_pcd, positions, 
                                                                    first_position, sampled_indices_global, chunk_size=chunk_size, 
                                                                    overlap=overlap, major_voxel_size=major_voxel_size,kitti_labels=kitti_labels_orig)
                
                print("finished downsample")
                
                
                patchwise_indices = indices_per_patch(T_pcd, center_positions, positions, first_position, sampled_indices_global, chunk_size)
                out_data = []
                for sequence in range(len(center_ids))[:-1]:
                                
                                print("sequence",sequence)
                                merged_chunk,file_name, pcd_chunk, pcd_chunk_ground, inliers_ground = ncuts_chunk(dataset,indices,pcd_nonground_chunks,pcd_ground_chunks,
                                        pcd_nonground_chunks_major_downsampling,
                                        pcd_nonground_minor,T_pcd,center_positions,center_ids,
                                        positions,first_position,sampled_indices_global,
                                        chunk_size=chunk_size,major_voxel_size=major_voxel_size,
                                        alpha=alpha,beta=beta,gamma=gamma,theta=theta,
                                        proximity_threshold=proximity_threshold,
                                        out_folder=out_folder_ncuts,ground_mode=False,sequence=sequence,
                                        patchwise_indices=patchwise_indices)
                                
                                #kitti_labels['ground']['panoptic'][sequence] = kitti_labels['ground']['panoptic'][sequence][inliers_ground]
                                kitti_labels['ground']['instance'][sequence] = kitti_labels['ground']['instance'][sequence][inliers_ground]
                                
                                name = file_name.split('/')[-1]
                                o3d.io.write_point_cloud(file_name, pcd_chunk + pcd_chunk_ground , write_ascii=False, compressed=False, print_progress=False)
                                
                                
                                cur_name = name.split('.')[0]
                                
                                #import pdb; pdb.set_trace()
                                #kitti_chunk = color_pcd_by_labels(pcd_chunk,kitti_labels['nonground']['panoptic'][sequence].reshape(-1,),
                                #                        colors=colors,gt_labels=kitti_labels_orig['panoptic_nonground'])
                                
                                kitti_chunk_instance = color_pcd_by_labels(pcd_chunk,kitti_labels['nonground']['instance'][sequence].reshape(-1,),
                                                        colors=colors,gt_labels=kitti_labels_orig['instance_nonground'])
                                
                                gt_pcd = kitti_chunk_instance + pcd_chunk_ground
                                pred_pcd = merged_chunk  + pcd_chunk_ground
                                
                                unique_colors, labels_ncuts = np.unique(np.asarray(pred_pcd.colors), axis=0, return_inverse=True)
                                unique_colors, labels_kitti = np.unique(np.asarray(gt_pcd.colors),axis=0, return_inverse=True)
                                
                                pts = np.asarray(gt_pcd.points) 
                                points,labels_ncuts,labels_kitti = downsample_chunk(pts,
                                                                                        labels_ncuts,labels_kitti)
                                print("outut",data_store_folder + name.split('.')[0])
                                np.savez(data_store_folder + name.split('.')[0] +  '.npz',pts=points,ncut_labels=labels_ncuts,
                                        kitti_labels=labels_kitti,cluster_labels=np.zeros_like(labels_ncuts))
                                
                                del points, labels_ncuts, labels_kitti 
                                del pred_pcd, gt_pcd, kitti_chunk_instance, pcd_chunk, pcd_chunk_ground
                                gc.collect()
                                
                                
                                #kitti_chunk_instance_ground = color_pcd_by_labels(pcd_chunk_ground,kitti_labels['ground']['instance'][sequence].reshape(-1,))
                                #o3d.io.write_point_cloud(out_dbscan + name, pcd_dbscan + pcd_chunk_ground, write_ascii=False, compressed=False, print_progress=False)
                                #o3d.io.write_point_cloud(out_kitti + name, kitti_chunk + pcd_chunk_ground, write_ascii=False, compressed=False, print_progress=False)
                                #o3d.io.write_point_cloud(out_kitti_instance + name, kitti_chunk_instance + pcd_chunk_ground, write_ascii=False, compressed=False, print_progress=False)
                                
        
        
        
        


Sequence 3
STORE FOLDER pcd_preprocessed/3/
ind start 0
ind end 801
process poses
PatchWorkpp::PatchWorkpp() - INITIALIZATION COMPLETE
load and downsample points
(11894976, 3)
(9399815, 3)
(9399815, 3)
(53945359, 3)
done downsample
write pcds
PointCloud with 11894976 points.
write labels
load pcd
load data
pose downsample
chunk downsample
Downsampled from (293897, 3) to (6274, 3) points (non-ground)
Downsampled from (197827, 3) to (5027, 3) points (ground)
Downsampled from (319820, 3) to (6580, 3) points (non-ground)
Downsampled from (211266, 3) to (5098, 3) points (ground)
Downsampled from (349559, 3) to (6713, 3) points (non-ground)
Downsampled from (224048, 3) to (4926, 3) points (ground)
Downsampled from (351513, 3) to (6485, 3) points (non-ground)
Downsampled from (234626, 3) to (5134, 3) points (ground)
Downsampled from (332827, 3) to (5974, 3) points (non-ground)
Downsampled from (236510, 3) to (4949, 3) points (ground)
Downsampled from (353506, 3) to (6158, 3) points (non-groun

In [4]:
o3d.io.write_point_cloud(f'{out_folder}pcd_ground_minor{SEQUENCE_NUM}_{cur_idx}.pcd', pcd_ground_minor, write_ascii=False, compressed=False, print_progress=True)

True

In [5]:
def divide_indices_into_chunks(max_index, chunk_size=1000):
    chunks = []
    for start in range(0, max_index, chunk_size):
        end = min(start + chunk_size, max_index)
        chunks.append((start, end))
    return chunks

# Example usage
max_index = 4500  # Example maximum index
chunks = divide_indices_into_chunks(max_index)

for chunk in chunks:
    print(chunk)

(0, 1000)
(1000, 2000)
(2000, 3000)
(3000, 4000)
(4000, 4500)


Now we aggregate a large point cloud based on (ind_start, ind_end)

In [6]:
'''
pcd_new = o3d.geometry.PointCloud()
pts_num = 1000000
pcd_new.points = o3d.utility.Vector3dVector(np.asarray(pcd_nonground_minor.points)[:pts_num])

map_labelled = color_pcd_by_labels(pcd_new,\
                kitti_labels['panoptic_nonground'][:pts_num].reshape(-1,1))

o3d.visualization.draw_geometries([map_labelled])
#o3d.io.write_point_cloud('labelled_map07.pcd',map_labelled)
'''

"\npcd_new = o3d.geometry.PointCloud()\npts_num = 1000000\npcd_new.points = o3d.utility.Vector3dVector(np.asarray(pcd_nonground_minor.points)[:pts_num])\n\nmap_labelled = color_pcd_by_labels(pcd_new,                kitti_labels['panoptic_nonground'][:pts_num].reshape(-1,1))\n\no3d.visualization.draw_geometries([map_labelled])\n#o3d.io.write_point_cloud('labelled_map07.pcd',map_labelled)\n"

Now we subsample the poses based on a voxel_size

Now we can split the point cloud into chunks based on a tbd chunk_size

In [7]:

unique_colors, labels_ncuts = np.unique(np.asarray(merge.colors), axis=0, return_inverse=True)
unique_colors, labels_dbscan = np.unique(np.asarray(merge_dbscan.colors), axis=0, return_inverse=True)
unique_colors, labels_kitti = np.unique(np.asarray(merge_kitti_instance.colors),axis=0, return_inverse=True)



new_ncuts_labels = remove_semantics(labels_kitti,labels_ncuts)
new_dbscan_labels = remove_semantics(labels_kitti,labels_dbscan)


'''
metrics_ncuts = Metrics(name='ncuts')
metrics_dbscan = Metrics(name='dbscan')
metrics_test = Metrics(name='test')

metrics_ncuts.update_stats(new_ncuts_labels,labels_kitti)

metrics_dbscan.update_stats(new_dbscan_labels,labels_kitti)

metrics_dbscan.compute_all_aps()
metrics_ncuts.compute_all_aps()
'''
#merge_vis = color_pcd_by_labels(merge,labels)
#o3d.visualization.draw_geometries([merge_vis])


NameError: name 'merge' is not defined

In [ ]:
new_kitti_pcd = color_pcd_by_labels(merge_dbscan,new_ncuts_labels)
o3d.io.write_point_cloud(out_folder + "ncuts_instances_tarl.pcd",new_kitti_pcd, write_ascii=False, compressed=False, print_progress=False)
#new_dbscan_pcd = color_pcd_by_labels(merge_dbscan,new_dbscan_labels)
#o3d.io.write_point_cloud(out_folder + "dbscan_instances.pcd",new_dbscan_pcd, write_ascii=False, compressed=False, print_progress=False)


True

In [ ]:
metrics_ncuts = Metrics(name='ncuts')
metrics_dbscan = Metrics(name='dbscan')
metrics_test = Metrics(name='test')

metrics_ncuts.update_stats(new_ncuts_labels,labels_kitti)
#metrics_dbscan.update_stats(new_dbscan_labels,labels_kitti)
metrics_ncuts.compute_all_aps()
#metrics_dbscan.compute_all_aps()

In [ ]:
o3d.visualization.draw_geometries([new_kitti_pcd])

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [ ]:
DATASET_PATH = os.path.join('/Users/cedric/Datasets/semantic_kitti/')
SEQUENCE_NUM = 7

dataset = create_kitti_odometry_dataset(DATASET_PATH,SEQUENCE_NUM,ncuts_mode=False)


In [ ]:
from open3d.pipelines import registration
import numpy as np 
merge.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.5,max_nn=200))

for i in range(70,72):
	local_pcd = o3d.geometry.PointCloud()
	local_pcd.points = o3d.utility.Vector3dVector(dataset[i].point_cloud[:, :3])
	
	
	## label visualization 
	labeled_pcd = o3d.geometry.PointCloud()
	labeled_pcd.points = o3d.utility.Vector3dVector(dataset[i].point_cloud[:, :3])
	labeled_pcd.colors = o3d.utility.Vector3dVector(np.vstack([0,0,0] for i in range(np.asarray(labeled_pcd.points).shape[0])))
	panoptic_labels = dataset[i].panoptic_labels # semantics + panoptics combined 
	semantic_labels = dataset[i].semantic_labels
	instance_labels = dataset[i].instance_labels
	intensity = dataset[i].intensity
	
	transform = dataset.get_pose(i)
	local_pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.5,max_nn=200))
	reg_p2l = registration.registration_icp(local_pcd, merge, 0.9, transform, registration.TransformationEstimationPointToPlane(), registration.ICPConvergenceCriteria(max_iteration=1000))
	transform = reg_p2l.transformation
	local_pcd.transform(transform)
	
	local_pcd.normals = o3d.utility.Vector3dVector([])
	
	local_pcd.paint_uniform_color([0, 0, 0])
	
	
	
	
	colors, labels,local_labels = kDTree_1NN_feature_reprojection_colors(np.asarray(local_pcd.colors), local_pcd, np.asarray(merge.colors), merge,panoptic_labels, max_radius=0.2)
	labeled_pcd = color_pcd_by_labels(labeled_pcd,labels.reshape(-1,))
	local_pcd.colors = o3d.utility.Vector3dVector(colors)
	#o3d.io.write_point_cloud("test.pcd", local_pcd, write_ascii=False, compressed=False, print_progress=False)
	
	
	
	labeled_pcd.translate([0,120,0])
	o3d.visualization.draw_geometries([local_pcd,labeled_pcd])
	unique_colors, labels = np.unique(colors, axis=0, return_inverse=True)
	unseen_mask = np.all(colors == [1,0,0], axis=1)
	labels[unseen_mask] = -1
	street_mask = np.all(colors == [0,0,0], axis=1)
	labels[street_mask] = 1
	
	#np.savez('output_files/7_tarl/ ' + str(i) + '.npz',labels=labels)

	

	#np.savez('output_files/7_original/' + str(i) + '.npz',ncut_labels=local_labels,kitti_labels=labels,pts=np.asarray(local_pcd.points),           
	#intensities=intensity,kitti_labels_semantic=semantic_labels,kitti_labels_instance=instance_labels)


In [ ]:
import numpy as np 
with np.load('output_files/7_original/0.npz') as data:
            xyz = data['pts'].astype(np.float)
            labels = data['ncut_labels'].astype(np.int32)  
            kitti_labels = data['kitti_labels']
            intensity = data['intensities']
            
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(xyz)
pcd = color_pcd_by_labels(pcd,labels)
o3d.visualization.draw_geometries([pcd])
        